## HTRU2 training
 This notebook loads the HTRU2 identification dataset (https://archive.ics.uci.edu/dataset/372/htru2), preprocesses it and trains a simple model

In [2]:
import sys
import os
PROJ_DIR = os.path.realpath(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.join(PROJ_DIR,'src'))



In [5]:
import pandas as pd
df = pd.read_csv(os.path.join(PROJ_DIR, 'assets', 'data', 'HTRU_2.csv'), delimiter=',', header=None)

In [10]:
df[8].count()

17898

Load data from CSV and save it to a suitable format. This can be skipped if concrete_data.npz is in assets.

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def load_htru2(path):
    df = pd.read_csv(path, delimiter=',', header=None)

    labels = df[8]
    x = df.drop(columns=8)

    possible_labels = labels.unique().tolist()
    print(possible_labels)
    print(len(possible_labels))
    y = labels.map(lambda x: possible_labels.index(x))

    return x.to_numpy(), y.to_numpy()
    
x, y = load_htru2(os.path.join(PROJ_DIR, 'assets', 'data', 'HTRU_2.csv'))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

[0, 1]
2


In [14]:
# Save to assets
np.savez(os.path.join(PROJ_DIR, 'assets', 'data', 'htru2.npz'),\
        x_train=x_train,\
        x_test=x_test,\
        y_train=y_train,\
        y_test=y_test)

Load data from file

In [15]:
file_data = np.load(os.path.join(PROJ_DIR, 'assets', 'data', 'htru2.npz'))
x_train = file_data['x_train']
x_test = file_data['x_test']
y_train = file_data['y_train']
y_test = file_data['y_test']

Train a MLP model

In [19]:
import torch

MODEL_NEURONS = 100
MODEL_EPOCHS= 2000
MODEL_LR = 1.0e-1
MODEL_LABEL_NUM = len(np.unique(y_train))

class MLP(torch.nn.Module):
    def __init__(self, n_neurons):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(x_train.shape[1], n_neurons)
        self.ac1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_neurons, MODEL_LABEL_NUM)
        self.ac2 = torch.nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        logits = self.fc2(x)
        x = self.ac2(logits)
        return x

x_train_tensor = torch.tensor(x_train).float()
y_train_tensor = torch.tensor(y_train)
x_test_tensor = torch.tensor(x_test).float()
y_test_tensor = torch.tensor(y_test)

label_onehot = torch.zeros(y_train.shape[0], MODEL_LABEL_NUM)
label_onehot.scatter_(1, y_train_tensor.unsqueeze(1), 1)
class_weights = 1.0/label_onehot.mean(axis=0)

network = MLP(MODEL_NEURONS)
loss = torch.nn.BCELoss(weight=class_weights)
optimizer = torch.optim.Adam(network.parameters(), lr=MODEL_LR)#, weight_decay=5e-3)

for epoch in range(MODEL_EPOCHS):
    optimizer.zero_grad()
    
    preds = network(x_train_tensor)
    label_onehot = torch.zeros(y_train.shape[0], MODEL_LABEL_NUM)
    label_onehot.scatter_(1, y_train_tensor.unsqueeze(1), 1)
    loss_value = loss(preds, label_onehot)
    loss_value.backward()        
    optimizer.step()

    train_accuracy = (preds.argmax(dim=1) == y_train_tensor).float().mean() 

    test_preds = network.forward(x_test_tensor)        
    test_accuracy = (test_preds.argmax(dim=1) == y_test_tensor).float().mean() 
    print(f'Epoch {epoch}/{MODEL_EPOCHS} - Loss: {loss_value.item()} - Train accuracy: {train_accuracy} - Test accuracy: {test_accuracy}')  
    if test_accuracy > 0.75: # Undertrained
        break
    
print(test_accuracy.item())


Epoch 0/2000 - Loss: 4.888294696807861 - Train accuracy: 0.1776784509420395 - Test accuracy: 0.9094972014427185
0.9094972014427185


In [18]:
# Save model
torch.save(network.state_dict(), os.path.join(PROJ_DIR,'assets','models','htru2-mlp.pth'))